In [3]:
from extractors import gov_extract, local_extract, web_scrapping, twitter_extract
import extractors.config as config
import pandas as pd
import numpy as np
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

# Extract

The extraction process is given to components in extractors folder that use the config files in configs folder to extract data from apis, local files and web scrapping. This is useful because I can modularize the code and make them more scalable.

The complete process consist in use gov_extract, local_extract and web_scrapping to get date from the different data sources specified in the [README](../README.md) and finally loading the raw data in memory for later use in transfrom phase.

- web_scrapping: This module is in charge of scrap the web content of "El Tiempo" to get information about news of COVID-19
- gov_extract: This module bring via API from dato.gov.co data about daily cases and test do it every day.
- twitter_extract: This module use Twitter API to bring data from last 4 days of twitter with cetain query (specified in [config file](./configs/twitter_api.yaml)).
- local_extract: This module load data from local files downloaded from DANE and Banco de la Republica.

In [9]:
daily_cases, daily_test = gov_extract.load()
employe, inflation = local_extract.load()

In [7]:
news = web_scrapping.load()

404 Client Error: Not Found for url: https://www.eltiempo.com/buscar/88?q=covid&category=salud&publishedAt%5Bfrom%5D=20-02-01&publishedAt%5Buntil%5D=20-12-01&contentTypes%5B0%5D=article


In [8]:
news = reduce(lambda x, y: x + y, news)

In [4]:
tweets = twitter_extract.load()

INFO:searchtweets.result_stream:using bearer token for authentication
INFO:searchtweets.result_stream:paging; total requests read so far: 1
INFO:searchtweets.result_stream:paging; total requests read so far: 2
INFO:searchtweets.result_stream:paging; total requests read so far: 3
INFO:searchtweets.result_stream:paging; total requests read so far: 4
INFO:searchtweets.result_stream:paging; total requests read so far: 5
INFO:searchtweets.result_stream:ending stream at 542 tweets
INFO:searchtweets.result_stream:using bearer token for authentication
INFO:searchtweets.result_stream:paging; total requests read so far: 1
INFO:searchtweets.result_stream:paging; total requests read so far: 2
INFO:searchtweets.result_stream:paging; total requests read so far: 3
INFO:searchtweets.result_stream:paging; total requests read so far: 4
INFO:searchtweets.result_stream:ending stream at 478 tweets
INFO:searchtweets.result_stream:using bearer token for authentication
INFO:searchtweets.result_stream:paging; 

# Transform

Transforming the data bringing from the APIs and web scrapping divide in two phases, the first one is the traditional phase when I clean and transform the data and the second one when I create the mineable views crossing the data by specified key.

## Clean and transform the data

In [40]:
import itertools

# Create a list of years and months to transpose the employee data.
year_list = [x for x in range(2001, 2021)]
month_list = [x for x in range(1, 13)]

column_names = ['Concepto']
column_names = column_names + ['{}/{}/{}'.format(1, str(date[1]),str(date[0])) for date in itertools.product(year_list, month_list)]

# Eliminate the last two columns of the raw data.
column_names = column_names[:-2]

In [41]:
employe_processed = employe.dropna().copy()
column_dict = dict(zip(list(employe_processed.columns), column_names))

employe_processed.rename(columns = column_dict, inplace = True)

# Tranpose the raw data of employee data download from DANE, because the original data was like
# in axis 1 the dates and in axis 0 the values.
employe_processed = employe_processed.transpose()
employe_processed.drop(['Concepto'], inplace = True)
employe_processed.rename(
    columns = {
        12: "TGP",
        13: "TO",
        14: "TD",
        16: "Ocupados",
        17: "Desocupados",
        18: "Inactivos"
    }, inplace = True)

# Get the data from the last 8 months, to fit with daily positive cases and test.
employe_processed = employe_processed[-8:]
employe_processed.reset_index(inplace = True)
employe_processed.rename(columns = {'index': 'fecha'}, inplace = True)

employe_processed

,fecha,TGP,TO,TD,Ocupados,Desocupados,Inactivos
0,1/3/2020,60.1793,52.7533,12.3397,20859.9,2936.39,15746.1
1,1/4/2020,51.4751,41.2136,19.9349,16374,4076.86,19278.8
2,1/5/2020,55.3874,43.6973,21.1061,17375.4,4648.35,17739.3
3,1/6/2020,57.2966,45.4439,20.6866,18214.6,4750.77,17116.2
4,1/7/2020,57.2245,45.9524,19.698,18191.5,4462.35,16933.9
5,1/8/2020,59.2239,49.1646,16.9851,19608.7,4012,16263
6,1/9/2020,60.2381,50.5068,16.1547,20172.7,3886.71,15881.1
7,1/10/2020,60.9711,51.4164,15.6708,20626.3,3832.98,15656.9


In [42]:
# Drop unnecessary data from raw data.
inflation_processed = inflation[7:341].drop(['Unnamed: 5'], axis = 1).copy()
inflation_processed.rename(
    columns = {
        "Meta de inflación e inflación total al consumidor": "fecha",
        "Unnamed: 1": "Inflación total",
        "Unnamed: 2": "Límite superior",
        "Unnamed: 3": "Meta de inflación",
        "Unnamed: 4": "Límite inferior"
    }, inplace = True)
# Get the data from the last 8 months, to fit with daily positive cases and test.
inflation_processed = inflation_processed[:8].reset_index().drop(['index'], axis = 1)

inflation_processed['fecha'] = inflation_processed['fecha'].apply(lambda x: '{}/{}/{}'.format(1, int(str(x)[4:6]), str(x)[0:4]))
inflation_processed = inflation_processed.iloc[::-1]
inflation_processed

,fecha,Inflación total,Límite superior,Meta de inflación,Límite inferior
7,1/3/2020,3.86,4,3,2
6,1/4/2020,3.51,4,3,2
5,1/5/2020,2.85,4,3,2
4,1/6/2020,2.19,4,3,2
3,1/7/2020,1.97,4,3,2
2,1/8/2020,1.88,4,3,2
1,1/9/2020,1.97,4,3,2
0,1/10/2020,1.75,4,3,2


In [43]:
daily_test_processed = daily_test.drop([0], axis = 0).copy()

# Transform the raw data of column 'fecha' to common standard from all data.
daily_test_processed['fecha'] = daily_test_processed['fecha'].apply(lambda x: '{}/{}/{}'.format(int(x.split("-")[2][:2]), int(x.split("-")[1]), x.split("-")[0]))

# Drop unnecessary columns from raw data.
daily_test_processed.drop(['acumuladas', 'positivas_acumuladas', 'negativas_acumuladas', 'positividad_acumulada', 'indeterminadas'], axis = 1, inplace = True)
daily_test_processed.fillna(value = 0, inplace = True)

# Transpose data from custom way from raw data to obtain only 3 columns from original data, I only need "fecha", "cantidad" and "procedencia"
cities = list(list(daily_test_processed.columns)[1:])
daily_test_transposed = pd.DataFrame(columns = ['fecha', 'cantidad', 'procedencia'])

transposed_data = []
for index, row in daily_test_processed.iterrows():
    for _, city in enumerate(cities):
        transposed_data.append([row['fecha'], row[city], str.upper(city)])

transposed_data = np.array(transposed_data)
daily_test_transposed['fecha'] = transposed_data[:,0]
daily_test_transposed['cantidad'] = transposed_data[:,1]
daily_test_transposed['procedencia'] = transposed_data[:,2]

daily_test_transposed = daily_test_transposed.astype({'cantidad': 'float64'})
daily_test_transposed

,fecha,cantidad,procedencia
0,5/3/2020,0.0,AMAZONAS
1,5/3/2020,0.0,ANTIOQUIA
2,5/3/2020,0.0,ARAUCA
3,5/3/2020,0.0,ATLANTICO
4,5/3/2020,0.0,BOGOTA
...,...,...,...
10483,5/12/2020,12007.0,PROCEDENCIA_DESCONOCIDA
10484,5/12/2020,0.0,BARRANQUILA
10485,5/12/2020,182583.0,CARTAGENA
10486,5/12/2020,79422.0,SANTA_MARTA


In [44]:
daily_cases_processed = daily_cases.copy()
daily_cases_processed.reset_index(inplace = True)
daily_cases_processed.drop(['index', 'fecha_de_notificaci_n', 'id_de_caso', 'departamento', 'pais_viajo_1_cod',
                            'ciudad_municipio', 'unidad_medida', 'fecha_reporte_web', 'per_etn_'], 
                           axis =  1, inplace = True)

# Formate the date field in certain columns to be common to all data.
column_date_list = ['fecha_diagnostico', 'fecha_inicio_sintomas', 'fecha_recuperado', 'fecha_muerte']

for column in column_date_list:
    daily_cases_processed[column] = daily_cases_processed[column].apply(lambda x: '{}'.format(x.split(' ')[0]) if type(x) == str else x)

# Lowerilize the "recuperado" column because certain values are duplicated but with capital letters
daily_cases_processed['recuperado'] = daily_cases_processed['recuperado'].apply(lambda x: str.lower(x) if type(x) == str else x)

# Drop unnecessary data.
daily_cases_processed.drop(['nom_grupo_'], inplace = True, axis = 1)
daily_cases_processed.drop(['pais_viajo_1_nom'], inplace = True, axis = 1)

daily_cases_processed

,departamento_nom,ciudad_municipio_nom,edad,sexo,fuente_tipo_contagio,ubicacion,estado,recuperado,fecha_inicio_sintomas,fecha_muerte,fecha_diagnostico,fecha_recuperado,tipo_recuperacion
0,BOGOTA,BOGOTA,19,F,Importado,Casa,Leve,recuperado,27/2/2020,NaN,6/3/2020,13/3/2020,PCR
1,VALLE,BUGA,34,M,Importado,Casa,Leve,recuperado,4/3/2020,NaN,9/3/2020,19/3/2020,PCR
2,ANTIOQUIA,MEDELLIN,50,F,Importado,Casa,Leve,recuperado,29/2/2020,NaN,9/3/2020,15/3/2020,PCR
3,ANTIOQUIA,MEDELLIN,55,M,Relacionado,Casa,Leve,recuperado,6/3/2020,NaN,11/3/2020,26/3/2020,PCR
4,ANTIOQUIA,MEDELLIN,25,M,Relacionado,Casa,Leve,recuperado,8/3/2020,NaN,11/3/2020,23/3/2020,PCR
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352498,BOYACA,TUNJA,36,M,En estudio,Casa,Leve,activo,3/11/2020,NaN,17/11/2020,NaN,NaN
1352499,HUILA,NEIVA,47,M,En estudio,Casa,Leve,activo,25/10/2020,NaN,8/11/2020,NaN,NaN
1352500,BOYACA,TUNJA,27,M,En estudio,Casa,Leve,activo,24/10/2020,NaN,7/11/2020,NaN,NaN
1352501,BOYACA,TIBASOSA,41,F,En estudio,Casa,Leve,activo,24/10/2020,NaN,7/11/2020,NaN,NaN


In [45]:
import re
import string
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text

In [46]:
# Clean and reformat data from news scrapped from news "El Tiempo", scapping special charecters in text and reformating dates.
news_array = [[x['title'].replace('\n', ''), x['publised_time'].strftime('%-d/%-m/%Y'), x['category'], x['resume'].replace('\n', '')] for x in news]

news_df = pd.DataFrame(data = news_array, columns = ['titulo', 'fecha', 'categoria', 'resumen'])
news_df.drop_duplicates(subset = ["titulo", "resumen"], inplace = True)

# Reversing the ordered dataframe, to get order frame by date.
news_df = news_df.iloc[::-1]

news_df.reset_index(inplace = True)
news_df.drop(['index'], inplace = True, axis = 1)
news_df[['titulo', 'resumen']] = news_df[['titulo', 'resumen']].replace('', float('NaN'))
news_df.dropna(subset = ['titulo', 'resumen'], inplace = True)
news_df['titulo'] = news_df['titulo'].apply(lambda x: clean_text(x))
news_df['resumen'] = news_df['resumen'].apply(lambda x: clean_text(x))
news_df

,titulo,fecha,categoria,resumen
0,coronavirus tiene nombre oficial oms lo bauti...,11/2/2020,Salud,nace del acrónimo en inglés a partir de la exp...
1,el covid una amenaza mundial que deja más de...,11/2/2020,Salud,según advirtió la organización mundial de la s...
2,así sería la cuarentena en el país para los co...,18/2/2020,Salud,los connacionales que serán evacuados de chin...
3,no se ha confirmado ningún caso de coronavirus...,19/2/2020,Salud,directivas del centro hospitalario indicaron q...
4,así será la evacuación y llegada de los colomb...,21/2/2020,Salud,ministerio de salud sostiene que el viaje part...
...,...,...,...,...
861,minsalud dice que por ahora no pedirá pruebas ...,1/12/2020,Salud,cartera envió preguntas al juez que con tutela...
862,europa decide sobre primera vacuna covid ante...,1/12/2020,Salud,agencia europea de medicamentos ema puso pla...
863,colombia registra muertes más y nuevos caso...,1/12/2020,Salud,el ministerio de salud reportó también recup...
864,vacuna a toda costa reflexiones tras días de ...,1/12/2020,Salud,análisis de alejandro gaviria tatiana andia ...


In [47]:
# Clean and reformat data getting via API from Twitter API.
new_tweets = [[x['id'], x['text'], x['fecha']] if 'id' in x else ['nan', 'nan', 'nan'] for x in tweets]
tweets_df = pd.DataFrame(new_tweets, columns = ['id', 'text', 'fecha'])
tweets_df['fecha'] = tweets_df['fecha'].apply(lambda x: '{}/{}/{}'.format(int(x.split('-')[2]), int(x.split('-')[1]), int(x.split('-')[0])) if x != 'nan' else x)
tweets_df.drop_duplicates(subset = ['text'], inplace = True)

# Reversing the ordered dataframe, to get order frame by date.
tweets_df = tweets_df.iloc[::-1]

tweets_df.reset_index(inplace = True)
tweets_df.drop(['index'], inplace = True, axis = 1)
tweets_df['text'] = tweets_df['text'].apply(lambda x: clean_text(x))
tweets_df['fecha'] = tweets_df['fecha'].replace('nan', float('NaN'))
tweets_df.dropna(inplace = True)
tweets_df

,id,text,fecha
0,1334648858744414210,coronavirus covid covid colombia ale...,4/12/2020
1,1334660151844986881,camiloprietoval minsaludcol respónda a conci...,4/12/2020
2,1334661194909671424,coronavirus salud nacionales colombia ...,4/12/2020
3,1334662001554022402,colombia de recuperación y de mortal...,4/12/2020
4,1334664187860164609,colombia de recuperación y de mortal...,4/12/2020
...,...,...,...
869,1333921344996708352,somos tan vulnerables que venezuela es uno de...,1/12/2020
870,1333921455814438913,rt noticiasonu la pandemia de ha disparado...,1/12/2020
871,1333923117387943941,rt coronavirusnewv 🇨🇴 colombia nuevos ...,1/12/2020
872,1333923715466326022,entérate demandan a ministro de salud de ...,1/12/2020


# Create mineable views

Mineable views are for better and optimize way to make queries to a databases, to create a mineable view I cross the data by specified keys, the complete mineables views and database are conformed by:

### Dimensional Views
- dates: This view is a dimentional view for query by date and optimize the speed of group and sort queries.
- origins: This view is another dimensional view specified for daily cases and daily test, to optimize and group data.

### Another
- employment: This view is for answer the questions and hipotesis planted for this analisys, specified the employment evolution by month relationship to another views with date_id from mineable date view.
- economy: Evolution of economy in the last 8 months in Colombia, relationship with date_id key.
- news: News scrapped from el Tiempo and relation ship with employment, economy, tweets, tests and cases. 
- tweets: Tweets transformed and cleaned in transform, relationship with other views by date_id in mineable date view

The last 2 views of the mineable views are the most richest in all bringed data about COVID-19:

- tests: Data from daily test and relationship with cases view with date_id and origin_id, this improve speed and group performance.
- cases: Data from daily cases crossed with test and other mineable views with date_id and origin_id.

* Note (the schema of the views are [here](./database/create_postgres.sql))

#### Cross data by date with date_id in date mineable view

In [48]:
import datetime
import math

# Helper function to cross data by certain ids (date_id and origin_id)
def get_id(df, column, target):
    return [ int(target[x]) if x in target else x for x in df[column].tolist() ]

"""
Creation of dimentional mineable view for dates to cross data from other views.
"""
base_date = datetime.datetime.today()
dates = [ base_date - datetime.timedelta(days = x) for x in range(310) ]
dates.reverse()

data = [[date.strftime('%-d/%-m/%Y'), int(date.strftime('%-d')), int(date.strftime('%-m')), int(date.strftime('%Y')), math.ceil(float(int(date.strftime('%-m')) / 3))] 
        for date in dates ]

date_table = pd.DataFrame(data, columns = ['cal_date', 'day', 'month', 'year', 'qtr'])
date_dict = {k: v for v, k in enumerate(date_table['cal_date'].tolist())}
date_table

,cal_date,day,month,year,qtr
0,31/1/2020,31,1,2020,1
1,1/2/2020,1,2,2020,1
2,2/2/2020,2,2,2020,1
3,3/2/2020,3,2,2020,1
4,4/2/2020,4,2,2020,1
...,...,...,...,...,...
305,1/12/2020,1,12,2020,4
306,2/12/2020,2,12,2020,4
307,3/12/2020,3,12,2020,4
308,4/12/2020,4,12,2020,4


In [49]:
employe_cross = employe_processed.copy()
employe_cross['date_id'] = get_id(employe_cross, 'fecha', date_dict)
employe_cross.drop(['fecha'], inplace = True, axis = 1)
employe_cross

,TGP,TO,TD,Ocupados,Desocupados,Inactivos,date_id
0,60.1793,52.7533,12.3397,20859.9,2936.39,15746.1,30
1,51.4751,41.2136,19.9349,16374,4076.86,19278.8,61
2,55.3874,43.6973,21.1061,17375.4,4648.35,17739.3,91
3,57.2966,45.4439,20.6866,18214.6,4750.77,17116.2,122
4,57.2245,45.9524,19.698,18191.5,4462.35,16933.9,152
5,59.2239,49.1646,16.9851,19608.7,4012,16263,183
6,60.2381,50.5068,16.1547,20172.7,3886.71,15881.1,214
7,60.9711,51.4164,15.6708,20626.3,3832.98,15656.9,244


In [50]:
inflation_cross = inflation_processed.copy()
inflation_cross['date_id'] = get_id(inflation_cross, 'fecha', date_dict)
inflation_cross.drop(['fecha'], inplace = True, axis = 1)
inflation_cross

,Inflación total,Límite superior,Meta de inflación,Límite inferior,date_id
7,3.86,4,3,2,30
6,3.51,4,3,2,61
5,2.85,4,3,2,91
4,2.19,4,3,2,122
3,1.97,4,3,2,152
2,1.88,4,3,2,183
1,1.97,4,3,2,214
0,1.75,4,3,2,244


In [51]:
daily_test_cross = daily_test_transposed.copy()
daily_test_cross['date_id'] = get_id(daily_test_cross, 'fecha', date_dict)
daily_test_cross.drop(['fecha'], inplace = True, axis = 1)
daily_test_cross

,cantidad,procedencia,date_id
0,0.0,AMAZONAS,34
1,0.0,ANTIOQUIA,34
2,0.0,ARAUCA,34
3,0.0,ATLANTICO,34
4,0.0,BOGOTA,34
...,...,...,...
10483,12007.0,PROCEDENCIA_DESCONOCIDA,309
10484,0.0,BARRANQUILA,309
10485,182583.0,CARTAGENA,309
10486,79422.0,SANTA_MARTA,309


In [52]:
news_cross = news_df.copy()
news_cross['date_id'] = get_id(news_cross, 'fecha', date_dict)
news_cross.drop(['fecha'], inplace = True, axis = 1)

In [53]:
daily_cases_cross = daily_cases_processed.copy()
daily_cases_cross['date_symptoms_id'] = get_id(daily_cases_cross, 'fecha_inicio_sintomas', date_dict)
daily_cases_cross['date_dead_id'] = get_id(daily_cases_cross, 'fecha_muerte', date_dict)
daily_cases_cross['date_diagnosis_id'] = get_id(daily_cases_cross, 'fecha_diagnostico', date_dict)
daily_cases_cross['date_recovered_id'] = get_id(daily_cases_cross, 'fecha_recuperado', date_dict)

daily_cases_cross.fillna(value = {'date_symptoms_id': -1, 'date_dead_id': -1, 'date_diagnosis_id': -1, 'date_recovered_id': -1}, inplace = True)
daily_cases_cross = daily_cases_cross.astype({'date_symptoms_id': 'int', 'date_dead_id': 'int', 
                                              'date_diagnosis_id': 'int', 'date_recovered_id': 'int'})

daily_cases_cross.drop(['fecha_inicio_sintomas'], inplace = True, axis = 1)
daily_cases_cross.drop(['fecha_muerte'], inplace = True, axis = 1)
daily_cases_cross.drop(['fecha_diagnostico'], inplace = True, axis = 1)
daily_cases_cross.drop(['fecha_recuperado'], inplace = True, axis = 1)
daily_cases_cross

,departamento_nom,ciudad_municipio_nom,edad,sexo,fuente_tipo_contagio,ubicacion,estado,recuperado,tipo_recuperacion,date_symptoms_id,date_dead_id,date_diagnosis_id,date_recovered_id
0,BOGOTA,BOGOTA,19,F,Importado,Casa,Leve,recuperado,PCR,27,-1,35,42
1,VALLE,BUGA,34,M,Importado,Casa,Leve,recuperado,PCR,33,-1,38,48
2,ANTIOQUIA,MEDELLIN,50,F,Importado,Casa,Leve,recuperado,PCR,29,-1,38,44
3,ANTIOQUIA,MEDELLIN,55,M,Relacionado,Casa,Leve,recuperado,PCR,35,-1,40,55
4,ANTIOQUIA,MEDELLIN,25,M,Relacionado,Casa,Leve,recuperado,PCR,37,-1,40,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352498,BOYACA,TUNJA,36,M,En estudio,Casa,Leve,activo,NaN,277,-1,291,-1
1352499,HUILA,NEIVA,47,M,En estudio,Casa,Leve,activo,NaN,268,-1,282,-1
1352500,BOYACA,TUNJA,27,M,En estudio,Casa,Leve,activo,NaN,267,-1,281,-1
1352501,BOYACA,TIBASOSA,41,F,En estudio,Casa,Leve,activo,NaN,267,-1,281,-1


In [54]:
tweets_cross = tweets_df.copy()
tweets_cross['date_id'] = get_id(tweets_cross, 'fecha', date_dict)
tweets_cross.drop(['fecha'], inplace = True, axis = 1)
tweets_cross.drop(['id'], inplace = True, axis = 1)

#### Cross data by origin with origin_id in origin mineable view

In [55]:
# Clean daily test data of 'procedencia', because certain data came with cities instead of departaments
map_cities = {
     'BARRANQUILA': 'ATLANTICO',
     'CARTAGENA': 'ATLANTICO',
     'SANTA_MARTA': 'ATLANTICO',
     'BARRANQUILLA': 'ATLANTICO',
     'NARINO': 'NARIÑO',
     'NORTE_DE_SANTANDER': 'NORTE SANTANDER',
     'SAN_ANDRES': 'SAN ANDRES',
     'VALLE_DEL_CAUCA': 'PROCEDENCIA_DESCONOCIDA'
}

daily_test_cross['procedencia'] = daily_test_cross['procedencia'].apply(lambda x: map_cities[x] if x in map_cities else x)
daily_cases_cross['departamento_nom'] = daily_cases_cross['departamento_nom'].apply(lambda x: map_cities[x] if x in map_cities else x)

In [66]:
daily_test_cross['total'] = daily_test_cross.groupby(['procedencia', 'date_id'])['cantidad'].transform('sum')
daily_test_cross.drop_duplicates(subset=['procedencia', 'date_id'], inplace = True)

In [67]:
daily_test_cross['cantidad'] = daily_test_cross['total']
daily_test_cross.drop(['total'], axis = 1, inplace = True)
daily_test_cross

,cantidad,procedencia,date_id
0,0.0,AMAZONAS,34
1,0.0,ANTIOQUIA,34
2,0.0,ARAUCA,34
3,0.0,ATLANTICO,34
4,0.0,BOGOTA,34
...,...,...,...
10478,75684.0,SUCRE,309
10479,98597.0,TOLIMA,309
10480,385838.0,PROCEDENCIA_DESCONOCIDA,309
10481,3653.0,VAUPES,309


In [68]:
depts = daily_cases_cross['departamento_nom'].drop_duplicates().tolist()
depts.append('PROCEDENCIA_DESCONOCIDA')
daily_cases_location = pd.DataFrame(depts, columns = ['origin'])

In [69]:
dept_map = {k: v for v, k in enumerate(daily_cases_location['origin'].tolist())}

daily_test_cross['origin_id'] =  get_id(daily_test_cross, 'procedencia', dept_map)
daily_cases_cross['origin_id'] = get_id(daily_cases_cross, 'departamento_nom', dept_map)

daily_test_cross.drop(['procedencia'], inplace = True, axis = 1)
daily_cases_cross.drop(['departamento_nom', 'ciudad_municipio_nom'], inplace = True, axis = 1)

# Load

The load process of the data in a database was made using load script in sql with copy commands to load the data with the script one for local development [here](./database/load_postgres.sql) and other for production in Amazon RedShift [here](./database/load_redshift.sql). The SQL  script for mineable views for postgres and amazon redshift are in [create_postgres.sql](./database/create_postgres.sql) and [create_redshift.sql](./database/create_redshift.sql) respectively.

In [70]:
daily_test_cross.to_csv('./output/test.csv', index = True, sep = '|', index_label='test_id')
daily_cases_cross.to_csv('./output/cases.csv', index = True, sep = '|', index_label='case_id')
tweets_cross.to_csv('./output/tweets.csv', index = True, sep = '|', index_label='tweet_id')
news_cross.to_csv('./output/news.csv', index = True, sep = '|', index_label='news_id')
inflation_cross.to_csv('./output/economy.csv', index = True, sep = '|', index_label='economy_id')
employe_cross.to_csv('./output/employment.csv', index = True, sep = '|', index_label='employment_id')
daily_cases_location.to_csv('./output/locations.csv', index = True, sep = '|', index_label='origin_id')
date_table.to_csv('./output/date.csv', index = True, sep = '|', index_label='date_id')